# 2025: Day 1

In [1]:
%load_ext jupyter_ai_magics

In [32]:
import os
from getpass import getpass

import aocd
import numpy as np
import pandas as pd

In [3]:
if not os.getenv('AOC_SESSION'):
    os.environ['AOC_SESSION'] = getpass('AOC_SESSION:')

AOC_SESSION: ········


## Part 1

You arrive at the secret entrance to the North Pole base ready to start decorating. Unfortunately, the password seems to have been changed, so you can't get in. A document taped to the wall helpfully explains:

"Due to new security protocols, the password is locked in the safe below. Please see the attached document for the new combination."

The safe has a dial with only an arrow on it; around the dial are the numbers 0 through 99 in order. As you turn the dial, it makes a small click noise as it reaches each number.

The attached document (your puzzle input) contains a sequence of rotations, one per line, which tell you how to open the safe. A rotation starts with an L or R which indicates whether the rotation should be to the left (toward lower numbers) or to the right (toward higher numbers). Then, the rotation has a distance value which indicates how many clicks the dial should be rotated in that direction.

So, if the dial were pointing at 11, a rotation of R8 would cause the dial to point at 19. After that, a rotation of L19 would cause it to point at 0.

Because the dial is a circle, turning the dial left from 0 one click makes it point at 99. Similarly, turning the dial right from 99 one click makes it point at 0.

So, if the dial were pointing at 5, a rotation of L10 would cause it to point at 95. After that, a rotation of R5 could cause it to point at 0.

The dial starts by pointing at 50.

You could follow the instructions, but your recent required official North Pole secret entrance security training seminar taught you that the safe is actually a decoy. The actual password is the number of times the dial is left pointing at 0 after any rotation in the sequence.

For example, suppose the attached document contained the following rotations:

```
L68
L30
R48
L5
R60
L55
L1
L99
R14
L82
```

Following these rotations would cause the dial to move as follows:

- The dial starts by pointing at 50.
- The dial is rotated L68 to point at 82.
- The dial is rotated L30 to point at 52.
- The dial is rotated R48 to point at **0**.
- The dial is rotated L5 to point at 95.
- The dial is rotated R60 to point at 55.
- The dial is rotated L55 to point at **0**.
- The dial is rotated L1 to point at 99.
- The dial is rotated L99 to point at **0**.
- The dial is rotated R14 to point at 14.
- The dial is rotated L82 to point at 32.

Because the dial points at 0 a total of three times during this process, the password in this example is 3.

Analyze the rotations in your attached document. What's the actual password to open the door?

In [10]:
puzzle_1 = aocd.models.Puzzle(day=1, year=2025)
puzzle_1.examples[0].input_data

'L68\nL30\nR48\nL5\nR60\nL55\nL1\nL99\nR14\nL82'

In [19]:
def solve_part_1(puzzle_input: str) -> int:
    def parse(s: str) -> int:
        direction: s = s[0]
        value = int(s[1:])
        sign: int
        if direction == 'L':
            sign = -1
        elif direction == 'R':
            sign = 1
        else:
            raise ValueError(f"The line does not start with 'L' or 'R': {s}")
        return sign * value

    # Convert the input to a sequence of adjustments, starting at the initial position
    start_value = 50
    seq = np.array([start_value] + [parse(p) for p in puzzle_input.split()])
    # Apply the sequence of adjustments using cumulative sum.
    seq_values = seq.cumsum()
    # Apply modulo to determine the value considering the circular dial.
    mod_seq_values = np.mod(seq_values, 100)
    # Mark the zero values and count.
    zero_values = np.where(mod_seq_values == 0, 1, 0)
    return sum(zero_values).item()


solve_part_1(puzzle_1.examples[0].input_data)

3

In [20]:
solve_part_1(puzzle_1.input_data)

1182

## \--- Part Two ---

You're sure that's the right password, but the door won't open. You knock, but nobody answers. You build a snowman while you think.

As you're rolling the snowballs for your snowman, you find another security document that must have fallen into the snow:

"Due to newer security protocols, please use _password method 0x434C49434B_ until further notice."

You remember from the training seminar that "method 0x434C49434B" means you're actually supposed to count the number of times _any click_ causes the dial to point at `0`, regardless of whether it happens during a rotation or at the end of one.

Following the same rotations as in the above example, the dial points at zero a few extra times during its rotations:

*   The dial starts by pointing at `50`.
*   The dial is rotated `L68` to point at `82`; during this rotation, it points at `0` _once_.
*   The dial is rotated `L30` to point at `52`.
*   The dial is rotated `R48` to point at `_0_`.
*   The dial is rotated `L5` to point at `95`.
*   The dial is rotated `R60` to point at `55`; during this rotation, it points at `0` _once_.
*   The dial is rotated `L55` to point at `_0_`.
*   The dial is rotated `L1` to point at `99`.
*   The dial is rotated `L99` to point at `_0_`.
*   The dial is rotated `R14` to point at `14`.
*   The dial is rotated `L82` to point at `32`; during this rotation, it points at `0` _once_.

In this example, the dial points at `0` three times at the end of a rotation, plus three more times during a rotation. So, in this example, the new password would be `_6_`.

Be careful: if the dial were pointing at `50`, a single rotation like `R1000` would cause the dial to point at `0` ten times before returning back to `50`!

Using password method 0x434C49434B, _what is the password to open the door?_

In [99]:
np.floor_divide(np.array([50, 0, -50, -100]), 100)

array([ 0,  0, -1, -1])

In [140]:
def solve_part_2(puzzle_input: str) -> int:
    def parse(s: str) -> int:
        direction: s = s[0]
        value = int(s[1:])
        sign: int
        if direction == 'L':
            sign = -1
        elif direction == 'R':
            sign = 1
        else:
            raise ValueError(f"The line does not start with 'L' or 'R': {s}")
        return sign * value

    # Convert the input to a sequence of adjustments, starting at the initial position
    initial_value = 50
    adjustment = np.array([parse(p) for p in puzzle_input.split()])
    # Apply the sequence of adjustments using cumulative sum.
    end_value = initial_value + adjustment.cumsum()
    start_value = np.insert(
        end_value[0:-1],
        0,
        initial_value,
    )
    # # Calculate if the value results in a zero value
    start_at_zero = np.where(np.mod(start_value, 100) == 0, 1, 0)
    end_at_zero = np.where(np.mod(end_value, 100) == 0, 1, 0)
    # # Convert each value to a "century"
    century_start = np.floor(start_value / 100).astype(np.int32)
    century_end = np.floor(end_value / 100).astype(np.int32)
    # Calculate the changes in "century" for each adjustment
    century_change = np.abs(century_start - century_end)
    # Calculate the number of times the dial passed zero
    times_at_zero = np.where(
        start_value > end_value,
        century_change - start_at_zero + end_at_zero,  # Turned clockwise to the left.
        century_change,                                # Turned counter-clockwise to the right
    )
    df: pd.DataFrame = pd.DataFrame({
        'start_value': start_value,
        'adjustment': adjustment,
        'end_value': end_value,
        'start_at_zero': start_at_zero,
        'end_at_zero': end_at_zero,
        'century_start': century_start,
        'century_end': century_end,
        'century_change': century_change,
        'times_at_zero': times_at_zero,
    })
    return df, sum(df['times_at_zero'])



df, result = solve_part_2(puzzle_1.examples[0].input_data)
display(df)
result

,start_value,adjustment,end_value,start_at_zero,end_at_zero,century_start,century_end,century_change,times_at_zero
0,50,-68,-18,0,0,0,-1,1,1
1,-18,-30,-48,0,0,-1,-1,0,0
2,-48,48,0,0,1,-1,0,1,1
3,0,-5,-5,1,0,0,-1,1,0
4,-5,60,55,0,0,-1,0,1,1
5,55,-55,0,0,1,0,0,0,1
6,0,-1,-1,1,0,0,-1,1,0
7,-1,-99,-100,0,1,-1,-1,0,1
8,-100,14,-86,1,0,-1,-1,0,0
9,-86,-82,-168,0,0,-1,-2,1,1


6

In [141]:
df, result = solve_part_2(
    """L50
    R50
    L50
    R50
    L250
    L100"""
)
display(df)
result

,start_value,adjustment,end_value,start_at_zero,end_at_zero,century_start,century_end,century_change,times_at_zero
0,50,-50,0,0,1,0,0,0,1
1,0,50,50,1,0,0,0,0,0
2,50,-50,0,0,1,0,0,0,1
3,0,50,50,1,0,0,0,0,0
4,50,-250,-200,0,1,0,-2,2,3
5,-200,-100,-300,1,1,-2,-3,1,1


6

In [142]:
df, result = solve_part_2(puzzle_1.input_data)
# with pd.option_context('display.max_rows', None):
#     display(df)
result

6907